## Downloading OSM Network Data

Using osmnx and geopandas

1 - get a boundary to load the OSM
2 - download the graph
3 - extract nodes and get counts per node
4 - extract edges
5 - import into postgres

In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox

#north, south, east, west = 44.987, 42.528, -77.635, -81.144
north, south, east, west = 43.66695, 43.65178, -79.34404, -79.40960

# types of OSM filters
filter_drive = ('["area"!~"yes"]'
            '["highway"!~"motor|trunk|foot|proposed|construction|abandoned|platform|raceway|foot|cycleway"]'
            '["service"!~"driveway|emergency_access|parking_aisle|drive-through"]'
            '["access"!~"private"]')
filter_walk = ('["area"!~"yes"]'
            '["highway"!~"motor|trunk|proposed|construction|abandoned|platform|raceway"]'
            '["foot"!~"no"]'
            '["footway"!~"sidewalk"]'
            '["service"!~"private|parking_aisle"]'
            '["access"!~"private"]')

# downloading and projecting and simplyfing the graph
G = ox.graph_from_bbox(north, south, east, west, custom_filter=filter_drive)
G = ox.project_graph(G)



In [ ]:
ox.save_graphml(G, filename='network_road.graphml')
G = ox.load_graphml('network_road.graphml')

In [3]:
# grabbing all the nodes and put into a geodataframe
nodes_all = ox.graph_to_gdfs(G, edges=False)
nodes_all = nodes_all.to_crs(epsg=32617)

# count the number of edges per node and join to the nodes geodataframe
streets_per_node = ox.count_streets_per_node(G)
streets_per_node = pd.DataFrame.from_dict(data=streets_per_node,orient='index')
streets_per_node.columns = ["streets_n"]
nodes_all = nodes_all.join(streets_per_node)

# export nodes to a shp
nodes_all.to_file(driver = 'ESRI Shapefile', filename= "network_data/network_roads_nodes.shp")



# this could have been piped directly to a postgres database, but it was downloaded/processed 
# on a different server so it was easier to save as files and then import into a database

# generate edges, and put into a shp


In [12]:
edges_all = ox.graph_to_gdfs(G, nodes=False)
del edges_all["oneway"],edges_all["bridge"],edges_all["access"],edges_all["highway"]
del edges_all["junction"],edges_all["key"],edges_all["maxspeed"],edges_all["name"]
del edges_all["osmid"],edges_all["service"],edges_all["tunnel"],edges_all["width"],edges_all["lanes"]
#edges_all.head(50)
edges_all.to_file(driver = 'ESRI Shapefile', filename= "network_data/network_roads_edges.shp")

In [ ]:
#ox.save_graphml(G, filename='network_road.graphml')
#G = ox.load_graphml('network_road.graphml')